In [1]:
import torch
import torch.nn as nn
from torchvision.transforms import transforms
import numpy as np
from torch.autograd import Variable
from torchvision.models import squeezenet1_1
import torch.functional as F
from io import open
import os
from PIL import Image
import pathlib
import glob
import cv2
from PIL import ImageFile

In [2]:
train_path='E:\\ml\\train'
pred_path='E:\\ml\\predict'

In [3]:
root = pathlib.Path(train_path)
classes=sorted([j.name.split('/')[-1] for j in root.iterdir()])

In [4]:
class ConvNet(nn.Module):
    def __init__(self, num_classes=13):
        super(ConvNet, self).__init__()
        
        #((w-f+2P)/s) +1
        
        # Convolutional Layers
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(num_features=32)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2)

        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(num_features=64)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2)

        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(num_features=128)
        self.relu3 = nn.ReLU()
        self.pool3 = nn.MaxPool2d(kernel_size=2)

        # Fully Connected Layers
        self.fc1 = nn.Linear(in_features=128 * 28 * 28, out_features=512)
        self.relu4 = nn.ReLU()
        self.fc2 = nn.Linear(in_features=512, out_features=256)
        self.relu5 = nn.ReLU()
        self.fc3 = nn.Linear(in_features=256, out_features=num_classes)

    def forward(self, input):
        output = self.conv1(input)
        output = self.bn1(output)
        output = self.relu1(output)
        output = self.pool1(output)

        output = self.conv2(output)
        output = self.bn2(output)
        output = self.relu2(output)
        output = self.pool2(output)

        output = self.conv3(output)
        output = self.bn3(output)
        output = self.relu3(output)
        output = self.pool3(output)

        output = output.view(-1, 128 * 28 * 28)

        output = self.fc1(output)
        output = self.relu4(output)
        output = self.fc2(output)
        output = self.relu5(output)
        output = self.fc3(output)

        return output

In [5]:
checkpoint=torch.load('best_checkpoint.model')
model=ConvNet(num_classes=13)
model.load_state_dict(checkpoint)
model.eval()

ConvNet(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu1): ReLU()
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu2): ReLU()
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu3): ReLU()
  (pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=100352, out_features=512, bias=True)
  (relu4): ReLU()
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (relu5): ReLU()
  (fc3): Linear(i

In [6]:
data_transforms = transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

In [7]:
def prediction(img_path,data_transforms):
    
    image=Image.open(img_path)
    
    image_tensor=data_transforms(image).float()
    
    image_tensor=image_tensor.unsqueeze_(0)
  
    input=Variable(image_tensor)
    
    output=model(input)
    
    index=output.data.numpy().argmax()
    
    pred=classes[index]
    
    return pred

In [8]:
images_path=glob.glob(pred_path+ '/*.jpg')

In [10]:
pred_dict={}

for i in images_path:
    pred_dict[i[i.rfind('/')+1:]] =prediction(i, data_transforms)

In [11]:
pred_dict

{'E:\\ml\\predict\\p (1).jpg': 'Bulbul',
 'E:\\ml\\predict\\p (10).jpg': 'Owl',
 'E:\\ml\\predict\\p (100).jpg': 'Penguin',
 'E:\\ml\\predict\\p (101).jpg': 'Eagle',
 'E:\\ml\\predict\\p (102).jpg': 'Woodpecker',
 'E:\\ml\\predict\\p (103).jpg': 'Eagle',
 'E:\\ml\\predict\\p (104).jpg': 'Eagle',
 'E:\\ml\\predict\\p (105).jpg': 'Dove',
 'E:\\ml\\predict\\p (106).jpg': 'Penguin',
 'E:\\ml\\predict\\p (107).jpg': 'Duck',
 'E:\\ml\\predict\\p (108).jpg': 'Duck',
 'E:\\ml\\predict\\p (109).jpg': 'Owl',
 'E:\\ml\\predict\\p (11).jpg': 'Dove',
 'E:\\ml\\predict\\p (110).jpg': 'Owl',
 'E:\\ml\\predict\\p (111).jpg': 'Ostrich',
 'E:\\ml\\predict\\p (112).jpg': 'Owl',
 'E:\\ml\\predict\\p (113).jpg': 'Peacock',
 'E:\\ml\\predict\\p (114).jpg': 'Ostrich',
 'E:\\ml\\predict\\p (115).jpg': 'Duck',
 'E:\\ml\\predict\\p (116).jpg': 'Eagle',
 'E:\\ml\\predict\\p (117).jpg': 'Macaw',
 'E:\\ml\\predict\\p (118).jpg': 'Macaw',
 'E:\\ml\\predict\\p (119).jpg': 'Eagle',
 'E:\\ml\\predict\\p (12).jpg': 'Pe